In [30]:
import numpy as np
import pandas as pd
from coxdev import (CoxDeviance,
                    _reversed_cumsums)

from simulate import (simulate_df, 
                      all_combos,
                      rng)

In [31]:
data = simulate_df(all_combos[-1], nrep=3, size=4)
data = data.reset_index().drop(columns='index')
data

,start,event,status
0,1.207244,1.972807,0
1,1.403160,2.563656,0
2,4.394491,8.654270,1
3,2.372759,4.019447,0
4,0.778747,2.149091,0
...,...,...,...
161,3.477170,4.039996,1
162,3.477170,7.153075,1
163,3.477170,4.627917,1
164,3.477170,4.178720,0


In [32]:
cox = CoxDeviance(event=data['event'],
                  start=data['start'],
                  status=data['status'],
                  tie_breaking='efron')
(data['event'] < data['start']).sum()

0

In [33]:
X = rng.standard_normal(data.shape[0])
(X_event, 
 X_start) = _reversed_cumsums(X, 
                              event_order=cox._event_order,
                              start_order=cox._start_order)

tmp = X_event[cox._first] - X_start[cox._event_map]
cumsum_diff = np.zeros_like(tmp)
cumsum_diff[cox._event_order] = tmp

In [34]:
by_hand = []
for i in range(data.shape[0]):
    val = X[(data['event'] >= data['event'].iloc[i]) & (data['start'] < data['event'].iloc[i])].sum()
    by_hand.append(val)
by_hand = np.array(by_hand)
assert np.allclose(by_hand * np.array(data['status']), cumsum_diff * np.array(data['status']))